In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# update working directory
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom')

In [ ]:
!pip install torchmetrics

In [ ]:
import os
import time
import torch.multiprocessing
from copy import deepcopy
from tqdm import tqdm
import shutil
import numpy as np
from sklearn.metrics import average_precision_score, accuracy_score

from parameters import Parameters
from train.trainer import Trainer
from deepfake_datasets.datasets import get_dataloader
from train.early_stopping import EarlyStopping
from train.validate import validate_detection, validate_fully_supervised_localization, validate_weakly_supervised_localization
from utils.utils import compute_mean_iou, compute_mean_ap, compute_mean_f1, compute_mean_acc_detection, compute_mean_ap_detection, compute_batch_iou, compute_batch_localization_f1, compute_batch_ap, compute_accuracy_detection, compute_average_precision_detection


In [ ]:
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
new_dataset_name = 'pluralistic'

zip_path = f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets_zip/{new_dataset_name}.zip'
new_content_root_path = f'/content/datasets/dolos_data/celebahq/fake/'

os.makedirs(new_content_root_path, exist_ok=True)

# move the dataset from drive to /content (SSD) for better performance in I/O
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_content_root_path)

In [ ]:
# set parameters
params = Parameters()

# set experiment name
params.experiment_name = 'training_vit_weakly_supervised'

# set backbone
params.arch = 'CLIP:ViT-L/14'

# create output dirs
params.create_output_dirs()

# uncomment lines below to set new datasets
# new_drive_root_path = '/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-main/datasets/dolos_data/celebahq/'
# new_root_path = '/content/dataset'
# new_root_path = '..\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'
# new_root_path = 'D:\\Python\\DeCLIP-main\\datasets\\dolos_data\\celebahq\\'


# move the dataset from drive to /content for better performance in I/O
# shutil.copytree(new_drive_root_path, new_root_path, dirs_exist_ok=True)

new_root_path = '/content/datasets/dolos_data/celebahq/'
new_dataset_name = 'pluralistic'

params.update_dolos_data_paths(new_root_path, new_dataset_name)


# fix the backbone - to train only the decoder
params.fix_backbone = True

# set the feature layer and the decoder type
params.feature_layer = 'layer20'
params.decoder_type = 'conv-20'

# set the batch size and num threads
params.batch_size = 64
params.num_threads = 8

# set the number of epochs
params.num_iter = 30

# set loss type
params.loss_type = 'bce'

# task
params.task_type = 'weakly_supervised_localization'
params.weakly_supervised_label_comparison_type = 'expansion'

for key, value in params.__dict__.items():
    print(f"{key}: {value}")

experiment_name: training_vit_weakly_supervised
task_type: weakly_supervised_localization
data_label: train
arch: CLIP:ViT-L/14
fix_backbone: True
weight_decay: 0.0
batch_size: 64
num_threads: 8
init_type: normal
init_gain: 0.02
train_dataset: pluralistic
decoder_type: conv-20
feature_layer: layer20
early_stop_epochs: 5
optim: adam
beta1: 0.9
lr: 0.001
show_loss_freq: 50
num_iter: 30
data_root_path: /content/datasets/dolos_data/celebahq
train_fake_path: /content/datasets/dolos_data/celebahq/fake/pluralistic/images/train
valid_fake_path: /content/datasets/dolos_data/celebahq/fake/pluralistic/images/valid
test_fake_path: /content/datasets/dolos_data/celebahq/fake/pluralistic/images/test
train_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/pluralistic/masks/train
valid_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/pluralistic/masks/valid
test_masks_ground_truth_path: /content/datasets/dolos_data/celebahq/fake/pluralistic/masks/test
train_real_pat

In [ ]:
# get the model, along with its trainer
model_trainer = Trainer(params)

In [ ]:
# get the dataloaders
train_loader = get_dataloader(params)

val_params = deepcopy(params)
val_params.data_label = 'valid'
val_loader = get_dataloader(val_params)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
# start the training loop
torch.cuda.empty_cache()
# model_trainer.compute_pos_weight_per_dataset(train_loader)
early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.001)
best_metric = 0.0 # iou for localization, ap for detection
print('Length of training set:', len(train_loader.dataset))
print('Length of validation set:', len(val_loader.dataset))
start_time = time.time()
for epoch in tqdm(range(params.num_iter)):
    print('Epoch:', epoch)

    epoch_loss = 0
    for data in train_loader:
        model_trainer.total_steps += 1

        model_trainer.set_input(data)
        model_trainer.optimize_parameters()

        if model_trainer.total_steps % params.show_loss_freq == 0:
            epoch_loss += model_trainer.loss.item()
            print(f'Train Loss: {model_trainer.loss.item():.4f} at step {model_trainer.total_steps} \t Iter time: {(time.time() - start_time) / model_trainer.total_steps:.2f}')


    epoch_loss /= len(train_loader.dataset)
    print(f'Average Train Loss: {epoch_loss:.4f}')

    # compute training metrics
    if params.task_type == 'fully_supervised_localization':
        compute_mean_iou(model_trainer.ious, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ious = []

        compute_mean_f1(model_trainer.f1_best, model_trainer.f1_fixed, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.f1_best = []
        model_trainer.f1_fixed = []

        compute_mean_ap(model_trainer.ap, verbose=True, extra_text=f'Train epoch {epoch} ')
        model_trainer.ap = []

    elif params.task_type == 'detection':
        model_trainer.format_output_detection()

        # compute_mean_acc_detection(model_trainer.logits, model_trainer.labels, verbose=True, extra_text=f'Train epoch {epoch} ')
        # compute_mean_ap_detection(model_trainer.logits, model_trainer.labels, verbose=True, extra_text=f'Train epoch {epoch} ')

        logits_np = model_trainer.logits.detach().cpu().numpy()
        labels_np = model_trainer.labels.detach().cpu().numpy()

        num_classes = logits_np.shape[1]
        labels_np = labels_np.astype(int)
        labels_one_hot = np.eye(num_classes)[labels_np]

        ap = average_precision_score(labels_one_hot, logits_np, average='macro')
        acc = accuracy_score(labels_np, np.argmax(logits_np, axis=1))

        print(f'Train epoch {epoch} Mean ACC: {ap:.4f}')
        print(f'Train epoch {epoch} Mean AP: {acc:.4f}')

        model_trainer.logits = []
        model_trainer.labels = []


    # validate the model
    print('Validation:')
    if params.task_type == 'fully_supervised_localization':
        ious, f1_best, f1_fixed, mean_ap, _ = validate_fully_supervised_localization(model_trainer.model, val_loader, params.train_dataset)

        # compute metrics
        mean_iou = compute_mean_iou(ious, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        mean_f1_best, mean_f1_fixed = compute_mean_f1(f1_best, f1_fixed, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        mean_ap = compute_mean_ap(mean_ap, verbose=True, extra_text=f'Validation at epoch {epoch} ')

        # save the model if the mean iou is improved
        if mean_iou > best_metric:
            best_metric = mean_iou
            model_trainer.save_model(f'best_localization_model_iou_{mean_iou:.4f}.pth')
            print(f'Best model saved at epoch {epoch}!')

        # check for early stopping
        early_stopping(mean_iou)

    elif params.task_type == 'detection':
        ap, acc, _ = validate_detection(model_trainer.model, val_loader)

        print(f'Validation at epoch {epoch} - AP: {ap:.4f}, Acc: {acc:.4f}')

        # save the model if the mean ap is improved
        if ap > best_metric:
            best_metric = ap
            model_trainer.save_model(f'best_detection_model_ap_{ap:.4f}.pth')
            print(f'Best model saved at epoch {epoch}!')

        # check for early stopping
        early_stopping(ap)

    elif params.task_type == 'weakly_supervised_localization':
        ap, acc, _ = validate_weakly_supervised_localization(model_trainer.model, val_loader, params.weakly_supervised_label_comparison_type, params.train_dataset)

        if ap is not None:
            print(f'Validation at epoch {epoch} - AP: {ap:.4f}, Acc: {acc:.4f}')

        # save the model if the mean ap is improved
        if ap is not None and ap > best_metric:
            best_metric = ap
            model_trainer.save_model(f'best_detection_model_ap_{ap:.4f}.pth')
            print(f'Best model saved at epoch {epoch}!')

        # check for early stopping
        early_stopping(ap)

    # check if early stopping is triggered
    if early_stopping.early_stop:
        print("Early stopping triggered")
        continue_training = model_trainer.adjust_learning_rate()
        if continue_training:
            print("Continuing training with a learning rate reduced by a factor of 10")
            early_stopping = EarlyStopping(patience=params.early_stop_epochs, verbose=True, delta=-0.002) # adjust the delta only once, otherwise stop completely
        else:
            print(f"Early stopping training at epoch {epoch}")
            break
    print()
    torch.cuda.empty_cache()

Length of training set: 9000
Length of validation set: 900


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0
Train Loss: 0.3174 at step 50 	 Iter time: 2.58
Train Loss: 0.3138 at step 100 	 Iter time: 2.56
Average Train Loss: 0.0001
Validation:
Length of dataset:  900



  3%|▎         | 1/30 [06:25<3:06:23, 385.64s/it]


Epoch: 1
Train Loss: 0.3135 at step 150 	 Iter time: 2.73
Train Loss: 0.3134 at step 200 	 Iter time: 2.68
Train Loss: 0.3134 at step 250 	 Iter time: 2.65
Average Train Loss: 0.0001
Validation:
Length of dataset:  900


...


Epoch: 2
Train Loss: 0.3133 at step 300 	 Iter time: 2.72
Train Loss: 0.3133 at step 350 	 Iter time: 2.69
Train Loss: 0.3133 at step 400 	 Iter time: 2.67
Average Train Loss: 0.0001
Validation:
Length of dataset:  900



 10%|█         | 3/30 [19:13<2:52:59, 384.44s/it]


Epoch: 3


 10%|█         | 3/30 [19:44<2:57:42, 394.91s/it]


KeyboardInterrupt: 

In [ ]:
# model_dir = '/content/experiments/training_vit_weakly_supervised/models/'
# model_files = sorted(
#     [f for f in os.listdir(model_dir) if f.endswith('.pth')]
# )

# best_model = model_files[-1]
# best_model_path = os.path.join(model_dir, best_model)

In [ ]:
# shutil.copy(best_model_path, f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/weakly_supervised/{os.path.basename(best_model_path)}')

In [ ]:
model_trainer.save_model(f'best_weakly_supervised_model_epoch_3_expansion_pluralistic.pth')

In [ ]:
last_model_path = '/content/experiments/training_vit_weakly_supervised/models/best_weakly_supervised_model_epoch_3_expansion_pluralistic.pth'
shutil.copy(last_model_path, f'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/weakly_supervised/{os.path.basename(last_model_path)}')

'/content/drive/MyDrive/Colab Notebooks/Licenta/DeCLIP-custom/trained_models/weakly_supervised/best_weakly_supervised_model_epoch_3_expansion_pluralistic.pth'